In [7]:
import shapely
import rasterio
import sys,os,os.path

# Rasterio python expect ssl certs in Centos location
#os.environ['CURL_CA_BUNDLE']='/etc/ssl/certs/ca-certificates.crt'

In [8]:
from stac_tools.stac_item import spacenetStacItem



def createLinkList(stac_self, parent_catalog_loc, collection_loc):

    linkDict = [
            { "rel":"self", "href": stac_self},
            { "rel":"parent", "href": parent_catalog_loc, "type": "text/html"},
            { "rel":"collection", "href": collection_loc}
        ]
    
    
    return linkDict


def createAssetDict(thumbnail=[],
                   IMD=[],
                   IMD_XML=[],
                   PAN=[],
                   MUL=[],
                   MUL_PanSharpen=[],
                   RGB_PanSharpen=[],
                   RGBIR_PanSharpen=[]
                   ):
    assetDict = {}
    
    if thumbnail:
        assetDict.update({
            "thumbnail": {
              "href": thumbnail,
              "type": "image/jpeg"
            }
        }
        )
    if IMD:
        assetDict.update({
        "imd": {
          "href": IMD,
        }
        }
        )
        
    if IMD_XML:
        assetDict.update({
            "imd_xml": {
                      "href": IMD_XML,
                      "type": "application/xml"
                    }
        }
        )
    
    if PAN:
        assetDict.update({
    "PAN": {
      "href": PAN,
      "type": "image/vnd.stac.geotiff; cloud-optimized=true",
      "eo:bands": [8]
    }
            })
        
    if MUL:
        assetDict.update({
                "MUL": {
                  "href": MUL,
                  "type": "image/vnd.stac.geotiff; cloud-optimized=true",
                  "eo:bands": [0,1,2,3,4,5,6,7]
                }
            })
    
    if MUL_PanSharpen:
        assetDict.update({
            "MUL-PanSharpen": {
                  "href": MUL_PanSharpen,
                  "type": "image/vnd.stac.geotiff; cloud-optimized=true",
                  "eo:bands": [0,1,2,3,4,5,6,7]
                }
        }
        )
        
    if RGB_PanSharpen:
        assetDict.update({
            "RGB-PanSharpen": {
              "href": RGB_PanSharpen,
              "type": "image/vnd.stac.geotiff; cloud-optimized=true",
              "eo:bands": [4,2,1]
            }
        }
        )
    
    if RGBIR_PanSharpen:
        assetDict.update({
            "RGBIR-PanSharpen": {
              "href": RGBIR_PanSharpen,
              "type": "image/vnd.stac.geotiff; cloud-optimized=true",
              "eo:bands": [4,2,1,6]
            }
        }
        )
        
    return assetDict



def writeStac_Item(out_file,
                   title,
                   stac_path,
               parent_catalog,
               collection_path,
                   cog_path,
                   imd_path, 
                   pan_path,
                   mul_path,
                   mul_ps=[],
                   rgb_ps=[],
                   rgbir_ps=[],
                   thumbnail_path=[], 
                   idStr=[]):
    
    
    
    assetDict = createAssetDict(thumbnail=thumbnail_path,
                   IMD=imd_path,
                   IMD_XML=imd_path,
                   PAN=pan_path,
                   MUL=mul_path,
                   MUL_PanSharpen=mul_ps,
                   RGB_PanSharpen=rgb_ps,
                   RGBIR_PanSharpen=rgbir_ps
                   )
    
    linkDict = createLinkList(stac_path, parent_catalog, collection_path)
    
    
    
    
    
    if idStr:
        pass
    else: 
        idStr = os.path.splitext(os.path.basename(imd_path))[0]
    
    stac_Item = spacenetStacItem(rasterPath=cog_path,
                                 title='',
                                 provider='DigitalGlobe', 
                                 license="CC 4.0 BY-SA", 
                                idStr=idStr, 
                                 assetDict=assetDict, 
                                 imdPath=imd_path, 
                                 links=linkDict)
    
    stac_Item.write_toJSON(out_file)

    
    return stac_Item
localSave = "../spacenet-stac/spacenet-dataset/"

In [9]:
%%time

## AOI 2 Vegas

spacenetStacLocation = "https://s3.amazonaws.com/spacenet-stac/"


collection_path = 'https://s3.amazonaws.com/spacenet-stac/spacenet-collections/spacenet-WV3-2A-collection.json'
parent_catalog = "https://s3.amazonaws.com/spacenet-stac/spacenet-challenge.json"
cog_path = "s3://spacenet-dataset/AOI_2_Vegas/srcData/rasterData/AOI_2_Vegas_MUL-PanSharpen_Cloud.tif"
pan_path = "s3://spacenet-dataset/AOI_2_Vegas/srcData/rasterData/AOI_2_Vegas_PAN_Cloud.tif"
mul_path = "s3://spacenet-dataset/AOI_2_Vegas/srcData/rasterData/AOI_2_Vegas_MUL_Cloud.tif"
mul_ps   = cog_path
rgb_ps   = "s3://spacenet-dataset/AOI_2_Vegas/srcData/rasterData/AOI_2_Vegas_RGB-PanSharpen_Cloud.tif"

stac_path = spacenetStacLocation + "spacenet-dataset/AOI_2_Vegas.json"
imd_path = 's3://spacenet-dataset/AOI_2_Vegas/srcData/rasterData/Vegas_Imagery/15OCT22183656-M2AS-055649178030_01_P001.XML'
thumbnail_path = 's3://spacenet-dataset/AOI_2_Vegas/srcData/rasterData/Vegas_Imagery/15OCT22183656-M2AS-055649178030_01_P001-BROWSE.JPG'
out_file = localSave + os.path.splitext(os.path.basename(cog_path))[0]+".json"
title = 'SpaceNet AOI 2 Las Vegas'


writeStac_Item(out_file,
               title,
               stac_path,
               parent_catalog,
               collection_path,
                   cog_path,
                   imd_path, 
                   pan_path,
                   mul_path,
                   mul_ps=mul_ps,
                   rgb_ps=rgb_ps,
                   rgbir_ps=[],
                   thumbnail_path=thumbnail_path, 
                   idStr=[])



RasterioIOError: '/vsis3/spacenet-dataset/AOI_2_Vegas/srcData/rasterData/AOI_2_Vegas_MUL-PanSharpen_Cloud.tif' does not exist in the file system, and is not recognized as a supported dataset name.